# 数据处理

因为解析数据时考虑不充分，所以写入文件的数据需要经过处理。<br>
原数据存在的问题：
* 电影民称字段有有无关字段
* 当时写入文件时是csv文件，但是有些短评是英文的也有英文的逗号
* 短评数据写入文件时是每一条短评信息一行，但是短评中包含换行符<br>

In [1]:
import re

In [2]:
# 读取短评数据
with open('comments.csv', 'r') as f:
    comments = f.read()
comments = comments.split('\n') # 以换行符分割

因为每条短评本身有换行符，需要将同一条短评的信息组合起来

In [7]:
comment_data = []
# 将属于同一条短评信息的元素组合起来添加到新的列表中
for item in comments:
    # 如果一个元素中能匹配'短评,https://www.douban.com/people'，说明这是一条短评信息的开始
    # 如果不能匹配说明这个元素跟上一条元素属于同一条短评信息
    if re.search(r' 短评,https://www.douban.com/people', item): 
        comment = item
        comment_data.append(comment) # 将该元素添加到新的列表中
    else: 
        comment_data.pop(-1) # 去掉列表中的上一个元素
        comment += item # 将上一个元素和该元素组合起来
        comment_data.append(comment) # 将组合后的信息添加到新的列表中
comment_data = list(set(comment_data)) # 去重

In [9]:
for item in comment_data:
    # 用正则表达式匹配
    s = re.search(r'(\w+ 短评,)(https://www.douban.com/people/.+/,)(.*,)(\d,)(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},)(.+)(,\d+)', item)
    if s:
        comment = list(s.groups()) # 将tuple转换成列表
        comment[0] = re.sub(r' 短评', '', s.group(1)) # 去掉电影名称中的子字符串 '短评'
        comment[5] = s.group(6).strip() # 去掉短评两边的空格
        # 将处理后的短评信息组合后写入文件
        with open('clean_data.csv', 'a') as f:
            f.write(''.join(comment))
            f.write('\n')